In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydrake.all
import os
from IPython.display import display

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw
import pydrake.multibody.jupyter_widgets
import pydrake.systems.jupyter_widgets
from ipywidgets import FloatSlider, Layout

In [ ]:
r = 0x8b
g = 0x60
b = 0xe4

In [ ]:
r/0xff

In [ ]:
g/0xff

In [ ]:
r/0xff

In [ ]:
def AddPlanarBin(plant):
    # Parse bin model
    parser = pydrake.multibody.parsing.Parser(plant)
    bin_ = parser.AddModelFromFile("models/planar_bin.sdf")
    
    # Weld bin to world
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("bin_base", bin_),
        RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [0,0,-0.015])
    )
    
def AddSimpleBox(plant, mass=0.1, mu=1.0, width=0.225, depth=0.15, height=0.03):
    box_instance = plant.AddModelInstance("box")
    box_body = plant.AddRigidBody(
        "box_body",
        box_instance,
        pydrake.multibody.tree.SpatialInertia(mass=mass, p_PScm_E=np.array([0., 0., 0.]),
        G_SP_E=pydrake.multibody.tree.UnitInertia.SolidBox(width, depth, height))
    )
    
    # Set up collision
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            box_body,
            RigidTransform(),
            pydrake.geometry.Box(width-0.001, depth-0.001, height=0.001),
            "box_body", pydrake.multibody.plant.CoulombFriction(mu, mu)
        )
        i=0
        for x in [-width/2.0, width/2.0]:
            for y in [-depth/2.0, depth/2.0]:
                for z in [-height/2.0, height/2.0]:
                    plant.RegisterCollisionGeometry(
                        box_body,
                        RigidTransform([x, y, z]),
                        pydrake.geometry.Sphere(radius=1e-7), f"contact_sphere{i}",
                        pydrake.multibody.plant.CoulombFriction(mu, mu))
                    i += 1
        plant.RegisterVisualGeometry(
            box_body,
            RigidTransform(),
            pydrake.geometry.Box(width, depth, height),
            "box_body",
            [0.4, 0.4, 0.4, 1.0]) # Set to be the same color as the bin
    
    
    plant.WeldFrames(
        plant.world_frame(),
        box_body.body_frame(),
        RigidTransform(RotationMatrix(), [0,0,height/2])
    )
    return box_instance

def AddPaper(plant, mass=0.01, mu=1.0, width=0.225, depth=0.15, height=0.001):
    paper_instance = plant.AddModelInstance("paper")
    paper_body = plant.AddRigidBody(
        "paper_body",
        paper_instance,
        pydrake.multibody.tree.SpatialInertia(mass=mass, p_PScm_E=np.array([0., 0., 0.]),
        G_SP_E=pydrake.multibody.tree.UnitInertia.SolidBox(width, depth, height))
    )
    
    # Set up collision
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            paper_body,
            RigidTransform(),
            pydrake.geometry.Box(width-0.001, depth-0.001, height=0.0001),
            "paper_body", pydrake.multibody.plant.CoulombFriction(mu, mu)
        )
        i=0
        for x in [-width/2.0, width/2.0]:
            for y in [-depth/2.0, depth/2.0]:
                for z in [-height/2.0, height/2.0]:
                    plant.RegisterCollisionGeometry(
                        paper_body,
                        RigidTransform([x, y, z]),
                        pydrake.geometry.Sphere(radius=1e-7), f"contact_sphere{i}",
                        pydrake.multibody.plant.CoulombFriction(mu, mu))
                    i += 1
        plant.RegisterVisualGeometry(
            paper_body,
            RigidTransform(),
            pydrake.geometry.Box(width, depth, height),
            "paper_body",
            [0.9, 0.9, 0.9, 1.0])
    
    return paper_instance, paper_body


def AddPointFinger(plant):
    mu = 1.0
    finger = plant.AddModelInstance("finger")
    false_body1 = plant.AddRigidBody("false_body1", finger, pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    finger_body = plant.AddRigidBody("body", finger, pydrake.multibody.tree.SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))
    shape = pydrake.geometry.Sphere(0.01)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_x", plant.world_frame(), plant.GetFrameByName("false_body1"), [1, 0, 0], -.3, .3))
    plant.AddJointActuator("finger_x", finger_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_z", plant.GetFrameByName("false_body1"), plant.GetFrameByName("body"), [0, 0, 1], 0.0, 0.5))
    finger_z.set_default_translation(0.25)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

class PointFingerForceControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorInputPort(
            "desired_contact_force", pydrake.systems.framework.BasicVector(2))
        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]

        desired_force = self.get_input_port(0).Eval(context)
        output.SetFromVector(-finger_mass*g - desired_force)

In [ ]:
# Initialize plant
builder = pydrake.systems.framework.DiagramBuilder()
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
AddPlanarBin(plant)
box = AddSimpleBox(plant)
paper_instance, paper_body = AddPaper(plant)

plant.Finalize()

# Set up visualizer
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
    builder, scene_graph, delete_prefix_on_load=False)
vis.vis['drake']['4'].delete()  # Only flush the gripper geometry.

contact_visualizer = builder.AddSystem(pydrake.systems.meshcat_visualizer.MeshcatContactVisualizer(
    vis, plant=plant, contact_force_radius=0.005, contact_force_scale=40.0))
builder.Connect(plant.get_contact_results_output_port(),
                contact_visualizer.GetInputPort("contact_results"))

diagram = builder.Build()
simulator = pydrake.systems.analysis.Simulator(diagram)

simulator.set_target_realtime_rate(.5)  # slow motion!
vis.start_recording()
simulator.AdvanceTo(5)
vis.stop_recording()
vis.vis['drake/contact_forces'].delete()  # TODO: make my recording include the contact forces!
vis.publish_recording()

